**Classifying Exoplanets: Exploring NASA's Kepler Space Observatory Dataset**

Space has always beckoned humanity to explore its depths. One of the most fascinating subjects in astronomical research is exoplanets, planets that orbit stars beyond our solar system. Examining exoplanets gives insight into planetary formation and evolution, critical information when searching for habitable planets.

The Kepler Space Observatory, a NASA space telescope for finding exoplanets, has analyzed thousands of planets, especially ones that are roughly Earth-sized and located within the habitable zones of their parent stars. From 2009 to 2018, Kepler revolutionized our understanding of extrasolar systems by cross-checking previous observations of exoplanets and labeling them as confirmed planets, candidates, or false positives.

Our primary question is: *Can we accurately classify celestial bodies as exoplanets based on their observed characteristics using the Kepler exoplanet dataset?*

Our project will analyze the NASA Kepler exoplanet dataset from the Kepler Telescope. This dataset contains details about celestial objects, including their radius, transit, stellar luminosity, and other essential attributes. By analyzing this dataset, we hope to develop a predictive model that discerns exoplanets from other extrasolar entities, furthering our understanding of the universe.

In [4]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

In [5]:

##Reading the data 
exoplanet <- read_csv("https://raw.githubusercontent.com/QuwackJ/dsci-100-group-37/main/Data/cumulative.csv")


##Selecting for our predictors
exoplanet_selected <- exoplanet |>
                        select(koi_disposition, koi_period, koi_depth, koi_duration, koi_impact)


tail(exoplanet_selected)

##splitting into training and testing data
exoplanet_split <- initial_split(exoplanet_selected, prop = 0.75, strata = koi_disposition)
training_data <- training(exoplanet_split)   
testing_data <- testing(exoplanet_split)



# COUNT of all CONFIRMED, FALSE POSITIVE, and CANDIDATE planets

Rows: 9564 Columns: 50
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (5): kepoi_name, kepler_name, koi_disposition, koi_pdisposition, koi_tc...
dbl (43): rowid, kepid, koi_score, koi_fpflag_nt, koi_fpflag_ss, koi_fpflag_...
lgl  (2): koi_teq_err1, koi_teq_err2

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


koi_disposition,koi_period,koi_depth,koi_duration,koi_impact
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
CANDIDATE,47.1096306,752.2,5.7410,1.230
FALSE POSITIVE,8.5898708,87.7,4.8060,0.765
FALSE POSITIVE,0.5276985,1579.2,3.2221,1.252
CANDIDATE,1.7398494,48.5,3.1140,0.043
FALSE POSITIVE,0.6814016,103.6,0.8650,0.147
FALSE POSITIVE,4.8560348,76.7,3.0780,0.134


Mean value of all predictors for the whole dataset


In [5]:
##MEAN value of all predictors for the whole dataset
summary_table <- training_data |>
                    select(-koi_disposition) |>
                    map_df(mean, na.rm = TRUE)
summary_table

koi_period,koi_depth,koi_duration,koi_impact
<dbl>,<dbl>,<dbl>,<dbl>
80.445,23356.05,5.641699,0.7256953


Mean value of all predictors for confirmed exoplanets

In [7]:
##MEAN value of all predictors for confirmed exoplanets
summary_table_confirmed <- training_data |>
                    filter(koi_disposition == "CONFIRMED") |>
                    select(-koi_disposition) |>
                    map_df(mean, na.rm = TRUE)
summary_table_confirmed

koi_period,koi_depth,koi_duration,koi_impact
<dbl>,<dbl>,<dbl>,<dbl>
26.95055,1140.939,4.309482,0.4208111


Mean value of all predictors for false positives that are not exoplanets

In [9]:
summary_table_false <- training_data |>
                    filter(koi_disposition == "FALSE POSITIVE") |>
                    select(-koi_disposition) |>
                    map_df(mean, na.rm = TRUE)
summary_table_false

koi_period,koi_depth,koi_duration,koi_impact
<dbl>,<dbl>,<dbl>,<dbl>
73.64063,44131.4,6.607775,0.9504213


## Methods

Using the NASA Kepler exoplanet dataset, we aim to perform **K-nearest neighbor classification** and predict the most probable physical explanation of a KOI (Kepler Object of Interest). Since our objective is simply to classify whether a celestial body is an exoplanet or not, without delving into specifics about the planet type, we focus on the features that are most indicative of a planet's presence. In this context, the most crucial columns to include in our model to leverage the data most directly associated to exoplanets would be:

### Categorical variable:
 - **koi_disposition:** Category of a KOI from the Exoplanet Archive and using Kepler Data. A KOI can be classified into:
     - **Confirmed**: is an exoplanet
     - **False Positive**: failed one or more tests
     - **Candidate**: passed all prior tests but does not mean that all tests have been conducted.
     - **Not dispositioned**: Not enough tests have been performed
     
### Predictor variables:
 - **koi_period (Orbital period):** The presence of a consistent orbital period is one of the primary methods for detecting exoplanets, especially with the transit method.
 - **koi_depth (Transit depth):** The amount by which the star's brightness drops during a transit (indicative of a planet passing in front of the star). A significant, repeatable transit depth is a strong signal of an exoplanet.
 - **koi_duration (Transit Duration):** Gives information about the size and speed of the orbiting body. Together with orbital period and transit depth, it helps confirm the presence of a planet and rules out other phenomena like binary stars.
 - **koi_impact (Impact Parameter):** Sky-projected distance of the planet's path across the stellar disc. A feasible impact parameter, in conjunction with other transit characteristics, can help confirm the transit signals are due to a planet.

We have included a **bar graph** to compare the total counts of all dispositions of the exoplanets in the dataset used. \
We also have some **scatter plots** comparing the different predictors we are using against one another.